In [1]:
import os
import pandas as pd
import pickle

In [2]:
# set the path for reading the Compustat lists
path = 'D:\\studyproject\\bankruptcy\\data\\compustat\\' # for win decomment this line
# path = '/Users/user/Documents/Bankruptcy/bankruptcy/data/compustat/' # for mac decomment this line

# set the path for reading the csv files
# --------------- for win ---------------
path_b = 'convertedCSVfiles\\bankrupt\\'
path_h = 'convertedCSVfiles\\healthy\\'
# --------------- for mac ---------------
# path_b = './convertedCSVfiles/bankrupt/'
# path_h = './convertedCSVfiles/healthy/'


****

# 1. Merging Bankrupt Companies Data

In [3]:
# read the bankrupt list
bankrupt = pd.read_csv(str(path + 'list_bankrupt.csv'), dtype=object)

# remove the glitch column
bankrupt = bankrupt.drop(['Unnamed: 0'], axis=1)

bankrupt.head()


,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK
0,001367,Amber Resources Company of C,08/31/2012,2.0,3AMBE,023184203,0000276750
1,002033,Fairchild Corp. (The),11/01/2011,2.0,FCHDQ,303698104,0000009779
2,004049,Constar International Inc,06/01/2011,2.0,CNSTQ,21036U206,0000029806
3,004352,Energy Conversion Devices In,09/04/2012,2.0,ENERQ,292659109,0000032878
4,004768,Fleetwood Enterprises Inc.,08/23/2010,2.0,FLTWQ,339099103,0000314132


## 1.1. ISIN

In [4]:
# ----------------------------------------------------------------------
# work with ISIN codes converted from CUSIP
# ----------------------------------------------------------------------
# set the path

# read the bankrupt companies CUSIP to ISIN convert-table csv file
bankrupt_csp2isn = pd.read_csv(path_b + 'bankrupt_csp2isn.csv', dtype=object)
bankrupt_csp2isn.rename(columns={'Unnamed: 0':'CUSIP'}, inplace=True)

print(bankrupt_csp2isn.head(), '\n\n')

# check the compatibility of CUSIP columns in two dataframe, before merging
for i in range(len(bankrupt)):
    if bankrupt.iloc[i][5] != bankrupt_csp2isn.iloc[i][0]:
        print('WARNING: There is inconsistency at row:', i)
        

# ----------------------------------------------------------------------
# work with ISIN codes converted from Ticker
# ----------------------------------------------------------------------
# read the bankrupt companies Ticker to ISIN convert-table csv file
bankrupt_tic2isn = pd.read_csv(path_b + 'bankrupt_tic2isn.csv', dtype=object)
bankrupt_tic2isn.rename(columns={'Unnamed: 0':'Ticker'}, inplace=True)

print(bankrupt_tic2isn.head())

# check the compatibility of Ticker columns in two dataframe, before merging
for i in range(len(bankrupt)):
    if bankrupt.iloc[i][4] != bankrupt_tic2isn.iloc[i][0]:
        print('WARNING: There is inconsistency at row:', i)
        

       CUSIP          ISIN error
0  023184203  US0231842032   NaN
1  303698104  US3036981047   NaN
2  21036U206  US21036U2069   NaN
3  292659109  US2926591098   NaN
4  339099103  US3390991038   NaN 


  Ticker          ISIN                    error
0  3AMBE           NaN  No best match available
1  FCHDQ  US3036981047                      NaN
2  CNSTQ           NaN  No best match available
3  ENERQ  US2926591098                      NaN
4  FLTWQ  US3390991038                      NaN


In [5]:
# attach the ISIN columns to the bankrupt dataframe
bankrupt['csp2ISIN'] = bankrupt_csp2isn['ISIN']
bankrupt['tic2ISIN'] = bankrupt_tic2isn['ISIN']

bankrupt.head()


,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK,csp2ISIN,tic2ISIN
0,001367,Amber Resources Company of C,08/31/2012,2.0,3AMBE,023184203,0000276750,US0231842032,NaN
1,002033,Fairchild Corp. (The),11/01/2011,2.0,FCHDQ,303698104,0000009779,US3036981047,US3036981047
2,004049,Constar International Inc,06/01/2011,2.0,CNSTQ,21036U206,0000029806,US21036U2069,NaN
3,004352,Energy Conversion Devices In,09/04/2012,2.0,ENERQ,292659109,0000032878,US2926591098,US2926591098
4,004768,Fleetwood Enterprises Inc.,08/23/2010,2.0,FLTWQ,339099103,0000314132,US3390991038,US3390991038


In [6]:
# ----------------------------------------------------------------------
# merge the two ISIN columns
# ----------------------------------------------------------------------
# add a single column for ISIN, and
bankrupt['ISIN'] = ''
# a column to indicate there was inconsistency in converting symbols
bankrupt['ISINc'] = ''

for index, row in bankrupt.iterrows():
    # in case of conflicts
    if row['csp2ISIN'] != row['tic2ISIN']:
        
        # in case csp2ISN is NAN
        if row['csp2ISIN'] != row['csp2ISIN']:
            # in case both are NAN values
            if row['tic2ISIN'] != row['tic2ISIN']:
                # enter NAN as the ISIN value
                row['ISIN'] = row['csp2ISIN']
                # but no real inconsistency
                row['ISINc'] = 0
                
            # in case only csp2ISIN is NAN
            else:
                # fill ISIN with tic2ISIN
                row['ISIN'] = row['tic2ISIN']
                # and mark it as an inconsistency
                row['ISINc'] = 1
        
        # in case csp2ISIN is non-NAN value
        else:
            # in case tic2ISIN is NAN
            if row['tic2ISIN'] != row['tic2ISIN']:
                # fill ISIN with csp2ISIN
                row['ISIN'] = row['csp2ISIN']
                # and mark it as an inconsistency
                row['ISINc'] = 1
            
            # in case both are non-NAN values
            else:
                # fill ISIN with both
                row['ISIN'] = str(row['csp2ISIN']) + ' - ' + str(row['tic2ISIN'])
                # and mark it as a serious inconsistency
                row['ISINc'] = 3
            
    # in case of consistency        
    else:
        # fill ISIN with the value
        row['ISIN'] = row['csp2ISIN']
        # and it's consistent
        row['ISINc'] = 0

        
# check the number of inconsistency cases
inconsistency = len(bankrupt) - bankrupt['ISINc'].value_counts()[0]
print(inconsistency, 'case(s) of inconsistency!')
# check for serious cases of inconsistency
if 3 in bankrupt['ISINc'].value_counts().index:
    print(bankrupt['ISINc'].value_counts()[3], 'are serious!')
else:
    print('But none is serious.')
    
# check the number of successful conversion to RIC
print('\nAnd now we have', bankrupt.count().ISIN, 'bankrupt company with ISIN code.')


# remove the extra *ISIN columns
bankrupt = bankrupt.drop(['csp2ISIN', 'tic2ISIN'], axis=1)

bankrupt.head()


19 case(s) of inconsistency!
But none is serious.

And now we have 111 bankrupt company with ISIN code.


,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK,ISIN,ISINc
0,001367,Amber Resources Company of C,08/31/2012,2.0,3AMBE,023184203,0000276750,US0231842032,1
1,002033,Fairchild Corp. (The),11/01/2011,2.0,FCHDQ,303698104,0000009779,US3036981047,0
2,004049,Constar International Inc,06/01/2011,2.0,CNSTQ,21036U206,0000029806,US21036U2069,1
3,004352,Energy Conversion Devices In,09/04/2012,2.0,ENERQ,292659109,0000032878,US2926591098,0
4,004768,Fleetwood Enterprises Inc.,08/23/2010,2.0,FLTWQ,339099103,0000314132,US3390991038,0


## 1.2. RIC

In [7]:
# ----------------------------------------------------------------------
# work with RIC codes converted from CUSIP
# ----------------------------------------------------------------------
# read the bankrupt companies CUSIP to RIC convert-table csv file
bankrupt_csp2ric = pd.read_csv(path_b + 'bankrupt_csp2ric.csv', dtype=object)
bankrupt_csp2ric.rename(columns={'Unnamed: 0':'CUSIP'}, inplace=True)

print(bankrupt_csp2ric.head(), '\n\n')

# check the compatibility of CUSIP columns in two dataframe, before merging
for i in range(len(bankrupt)):
    if bankrupt.iloc[i][5] != bankrupt_csp2ric.iloc[i][0]:
        print('WARNING: There is inconsistency at row:', i)
        

# ----------------------------------------------------------------------
# work with RIC codes converted from Ticker
# ----------------------------------------------------------------------
# read the bankrupt companies Ticker to ISIN convert-table csv file
bankrupt_tic2ric = pd.read_csv(path_b + 'bankrupt_tic2ric.csv', dtype=object)
bankrupt_tic2ric.rename(columns={'Unnamed: 0':'Ticker'}, inplace=True)

print(bankrupt_tic2ric.head())

# check the compatibility of Ticker columns in two dataframe, before merging
for i in range(len(bankrupt)):
    if bankrupt.iloc[i][4] != bankrupt_tic2ric.iloc[i][0]:
        print('WARNING: There is inconsistency at row:', i)
        


       CUSIP  RIC                    error
0  023184203  NaN  No best match available
1  303698104  NaN  No best match available
2  21036U206  NaN  No best match available
3  292659109  NaN  No best match available
4  339099103  NaN  No best match available 


  Ticker           RIC                    error
0  3AMBE           NaN  No best match available
1  FCHDQ  FCHDQ.PK^K11                      NaN
2  CNSTQ           NaN  No best match available
3  ENERQ  ENERQ.PK^I12                      NaN
4  FLTWQ  FLTWQ.PK^H10                      NaN


In [8]:
# attach the RIC columns to the bankrupt dataframe
bankrupt['csp2RIC'] = bankrupt_csp2ric['RIC']
bankrupt['tic2RIC'] = bankrupt_tic2ric['RIC']

bankrupt.head()


,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK,ISIN,ISINc,csp2RIC,tic2RIC
0,001367,Amber Resources Company of C,08/31/2012,2.0,3AMBE,023184203,0000276750,US0231842032,1,NaN,NaN
1,002033,Fairchild Corp. (The),11/01/2011,2.0,FCHDQ,303698104,0000009779,US3036981047,0,NaN,FCHDQ.PK^K11
2,004049,Constar International Inc,06/01/2011,2.0,CNSTQ,21036U206,0000029806,US21036U2069,1,NaN,NaN
3,004352,Energy Conversion Devices In,09/04/2012,2.0,ENERQ,292659109,0000032878,US2926591098,0,NaN,ENERQ.PK^I12
4,004768,Fleetwood Enterprises Inc.,08/23/2010,2.0,FLTWQ,339099103,0000314132,US3390991038,0,NaN,FLTWQ.PK^H10


In [9]:
# ----------------------------------------------------------------------
# merge the two RIC columns
# ----------------------------------------------------------------------
# add a single column for RIC, and
bankrupt['RIC'] = ''
# a column to indicate there was inconsistency in converting symbols
bankrupt['RICc'] = ''


for index, row in bankrupt.iterrows():
    # in case of conflicts
    if row['csp2RIC'] != row['tic2RIC']:
        
        # in case csp2RIC is NAN
        if row['csp2RIC'] != row['csp2RIC']:
            # in case both are NAN values
            if row['tic2RIC'] != row['tic2RIC']:
                # enter NAN as the RIC value
                row['RIC'] = row['csp2RIC']
                # but no real inconsistency
                row['RICc'] = 0
                
            # in case only csp2RIC is NAN
            else:
                # fill RIC with tic2RIC
                row['RIC'] = row['tic2RIC']
                # and mark it as an inconsistency
                row['RICc'] = 1
        
        # in case csp2RIC is non-NAN value
        else:
            # in case tic2RIC is NAN
            if row['tic2RIC'] != row['tic2RIC']:
                # fill RIC with csp2RIC
                row['RIC'] = row['csp2RIC']
                # and mark it as an inconsistency
                row['RICc'] = 1
            
            # in case both are non-NAN values
            else:
                # fill RIC with both
                row['RIC'] = str(row['csp2RIC']) + ' - ' + str(row['tic2RIC'])
                # and mark it as a serious inconsistency
                row['RICc'] = 3
            
    # in case of consistency        
    else:
        # fill RIC with the value
        row['RIC'] = row['csp2RIC']
        # and it's consistent
        row['RICc'] = 0

        
# check the number of inconsistency cases
inconsistency = len(bankrupt) - bankrupt['RICc'].value_counts()[0]
print(inconsistency, 'case(s) of inconsistency!')
# check for serious cases of inconsistency
if 3 in bankrupt['RICc'].value_counts().index:
    print(bankrupt['RICc'].value_counts()[3], 'are serious!')
else:
    print('But none is serious.')

# check the number of successful conversion to RIC
print('\nAnd now we have', bankrupt.count().RIC, 'bankrupt company with RIC code.')


# remove the extra *ISIN columns
bankrupt = bankrupt.drop(['csp2RIC', 'tic2RIC'], axis=1)


bankrupt.head()


61 case(s) of inconsistency!
But none is serious.

And now we have 62 bankrupt company with RIC code.


,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK,ISIN,ISINc,RIC,RICc
0,001367,Amber Resources Company of C,08/31/2012,2.0,3AMBE,023184203,0000276750,US0231842032,1,NaN,0
1,002033,Fairchild Corp. (The),11/01/2011,2.0,FCHDQ,303698104,0000009779,US3036981047,0,FCHDQ.PK^K11,1
2,004049,Constar International Inc,06/01/2011,2.0,CNSTQ,21036U206,0000029806,US21036U2069,1,NaN,0
3,004352,Energy Conversion Devices In,09/04/2012,2.0,ENERQ,292659109,0000032878,US2926591098,0,ENERQ.PK^I12,1
4,004768,Fleetwood Enterprises Inc.,08/23/2010,2.0,FLTWQ,339099103,0000314132,US3390991038,0,FLTWQ.PK^H10,1


****

# 2. Merging Healthy Companies Data

In [28]:
# read the healthy list
healthy = pd.read_csv(str(path + 'list_healthy.csv'), dtype=object)

# remove the glitch column
healthy = healthy.drop(['Unnamed: 0'], axis=1)

healthy.head()


,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK
0,001004,AAR Corp,.,NaN,AIR,000361105,0000001750
1,001013,ADC Telecommunications Inc.,12/10/2010,1.0,ADCT,000886309,0000061478
2,001021,AFP Imaging Corp,09/15/2014,7.0,IWKS,001058205,0000319126
3,001034,Alpharma Inc.,12/31/2008,1.0,ALO.2,020813101,0000730469
4,001045,American Airlines Group Inc,.,NaN,AAL,02376R102,0000006201


## 2.1. ISIN

In [29]:
# ----------------------------------------------------------------------
# read and merge ISIN codes converted from CUSIP
# ----------------------------------------------------------------------
# read the healthy companies CUSIP to ISIN conversion csv files
healthy_csp2isn = [pd.read_csv(path_h + 'healthy_csp2isn{}.csv'.format(i+1), dtype=object) for i in range(6)]
# merge them
healthy_csp2isn = pd.concat([healthy_csp2isn[i] for i in range(6)])
# clean the resulted dataframe
healthy_csp2isn.rename(columns={'Unnamed: 0':'CUSIP', 'ISIN':'csp2ISIN'}, inplace=True)
# and remove that extra uninformative column
healthy_csp2isn = healthy_csp2isn.drop(['error'], axis=1)

# check the dataframe
print('\nThe CUSIP-to-ISIN conversion dataframe:')
print('\n', healthy_csp2isn.head())

# first-merge CUSIP-to-ISIN dataframe with the healthy list
healthy = healthy.merge(healthy_csp2isn, how='left', on=['CUSIP'])

print('\n---------------------- First Merge Done! -----------------------------\n')

# the first merge results
print('The merge result:\n')
print(healthy.head())

print('\n----------------------------------------------------------------------\n')


# ----------------------------------------------------------------------
# read and merge ISIN codes converted from Ticker
# ----------------------------------------------------------------------
# read the healthy companies Ticker to ISIN conversion csv files
healthy_tic2isn = [pd.read_csv(path_h + 'healthy_tic2isn{}.csv'.format(i+1), dtype=object) for i in range(6)]
# merge them
healthy_tic2isn = pd.concat([healthy_tic2isn[i] for i in range(6)])
# clean the resulted dataframe
healthy_tic2isn.rename(columns={'Unnamed: 0':'Ticker', 'ISIN':'tic2ISIN'}, inplace=True)
# and remove that extra uninformative column
healthy_tic2isn = healthy_tic2isn.drop(['error'], axis=1)

# check the dataframe
print('\nThe Ticker-to-ISIN conversion dataframe:')
print('\n', healthy_tic2isn.head())

# second merge Ticker-to-ISIN dataframe with the first-merged healthy list
healthy = healthy.merge(healthy_tic2isn, on=['Ticker'], how='left')

print('\n--------------------- Second Merge Done! -----------------------------\n')

# the second merge results
print('The merge result:')
print('\n\n', healthy.head())


# ------------------------------------------------------------------------------

# Drop duplicate columns resulted from the merge method
healthy = healthy.drop_duplicates(subset=None, keep='first', inplace=False)



The CUSIP-to-ISIN conversion dataframe:

        CUSIP      csp2ISIN
0  000361105  US0003611052
1  000886309  US0008863096
2  001058205  US0010582056
3  020813101  US0208131013
4  02376R102  US02376R1023

---------------------- First Merge Done! -----------------------------

The merge result:

  Identifier                      Company Data Deletion Date Deletion Reason  \
0     001004                     AAR Corp                  .             NaN   
1     001013  ADC Telecommunications Inc.         12/10/2010             1.0   
2     001021             AFP Imaging Corp         09/15/2014             7.0   
3     001034                Alpharma Inc.         12/31/2008             1.0   
4     001045  American Airlines Group Inc                  .             NaN   

  Ticker      CUSIP         CIK      csp2ISIN  
0    AIR  000361105  0000001750  US0003611052  
1   ADCT  000886309  0000061478  US0008863096  
2   IWKS  001058205  0000319126  US0010582056  
3  ALO.2  020813101  000073046

In [30]:
# ----------------------------------------------------------------------
# merge the two ISIN columns
# ----------------------------------------------------------------------
# add a single column for ISIN, and
healthy['ISIN'] = ''
# a column to indicate there was inconsistency in converting symbols
healthy['ISINc'] = ''

for index, row in healthy.iterrows():
    # in case of conflicts
    if row['csp2ISIN'] != row['tic2ISIN']:
        
        # in case csp2ISN is NAN
        if row['csp2ISIN'] != row['csp2ISIN']:
            # in case both are NAN values
            if row['tic2ISIN'] != row['tic2ISIN']:
                # enter NAN as the ISIN value
                row['ISIN'] = row['csp2ISIN']
                # but no real inconsistency
                row['ISINc'] = 0
                
            # in case only csp2ISIN is NAN
            else:
                # fill ISIN with tic2ISIN
                row['ISIN'] = row['tic2ISIN']
                # and mark it as an inconsistency
                row['ISINc'] = 1
        
        # in case csp2ISIN is non-NAN value
        else:
            # in case tic2ISIN is NAN
            if row['tic2ISIN'] != row['tic2ISIN']:
                # fill ISIN with csp2ISIN
                row['ISIN'] = row['csp2ISIN']
                # and mark it as an inconsistency
                row['ISINc'] = 1
            
            # in case both are non-NAN values
            else:
                # fill ISIN with both
                row['ISIN'] = str(row['csp2ISIN']) + ' - ' + str(row['tic2ISIN'])
                # and mark it as a serious inconsistency
                row['ISINc'] = 3
            
    # in case of consistency        
    else:
        # fill ISIN with the value
        row['ISIN'] = row['csp2ISIN']
        # and it's consistent
        row['ISINc'] = 0

        
# check the number of inconsistency cases
inconsistency = len(healthy) - healthy['ISINc'].value_counts()[0]
print(inconsistency, 'case(s) of inconsistency!')
# check for serious cases of inconsistency
if 3 in healthy['ISINc'].value_counts().index:
    print(healthy['ISINc'].value_counts()[3], 'are serious!')
else:
    print('But none is serious.')
    
# check the number of successful conversion to ISIN
print('\nAnd now we have', len(healthy) - healthy['ISIN'].isna().sum(), 'bankrupt company with ISIN code.')


# remove the extra *ISIN columns
healthy = healthy.drop(['csp2ISIN', 'tic2ISIN'], axis=1)


healthy.head()


8135 case(s) of inconsistency!
204 are serious!

And now we have 19580 bankrupt company with ISIN code.


,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK,ISIN,ISINc
0,001004,AAR Corp,.,NaN,AIR,000361105,0000001750,US0003611052,0
1,001013,ADC Telecommunications Inc.,12/10/2010,1.0,ADCT,000886309,0000061478,US0008863096,1
2,001021,AFP Imaging Corp,09/15/2014,7.0,IWKS,001058205,0000319126,US0010582056,0
3,001034,Alpharma Inc.,12/31/2008,1.0,ALO.2,020813101,0000730469,US0208131013,1
4,001045,American Airlines Group Inc,.,NaN,AAL,02376R102,0000006201,US02376R1023,0


## 2.2. RIC

In [31]:
# ----------------------------------------------------------------------
# read and merge RIC codes converted from CUSIP
# ----------------------------------------------------------------------
# read the healthy companies CUSIP to RIC convert-table csv files
healthy_csp2ric = [pd.read_csv(path_h + 'healthy_csp2ric{}.csv'.format(i+1), dtype=object) for i in range(6)]
# merge them
healthy_csp2ric = pd.concat([healthy_csp2ric[i] for i in range(6)])
# clean the resulted dataframe
healthy_csp2ric.rename(columns={'Unnamed: 0':'CUSIP', 'RIC':'csp2RIC'}, inplace=True)
# and remove that extra uninformative column
healthy_csp2ric = healthy_csp2ric.drop(['error'], axis=1)

# check the dataframe
print('\nThe CUSIP-to-RIC conversion dataframe:')
print('\n', healthy_csp2ric.head())

# first-merge CUSIP-to-RIC dataframe with the healthy list
healthy = healthy.merge(healthy_csp2ric, how='left', on=['CUSIP']) 
# Drop duplicate columns resulted from the merge method
healthy = healthy.drop_duplicates(subset=None, keep='first', inplace=False)

print('\n---------------------- First Merge Done! -----------------------------\n')

# the first merge results
print('The merge result:\n')
print(healthy.head())

print('\n----------------------------------------------------------------------\n')


# ----------------------------------------------------------------------
# read and merge RIC codes converted from Ticker
# ----------------------------------------------------------------------
# read the healthy companies Ticker to RIC conversion csv files
healthy_tic2ric = [pd.read_csv(path_h + 'healthy_tic2ric{}.csv'.format(i+1), dtype=object) for i in range(6)]
# merge them
healthy_tic2ric = pd.concat([healthy_tic2ric[i] for i in range(6)])
# clean the resulted dataframe
healthy_tic2ric.rename(columns={'Unnamed: 0':'Ticker', 'RIC':'tic2RIC'}, inplace=True)
# and remove that extra uninformative column
healthy_tic2ric = healthy_tic2ric.drop(['error'], axis=1)

# check the dataframe
print('\nThe Ticker-to-RIC conversion dataframe:')
print('\n', healthy_tic2ric.head())

# second merge Ticker-to-RIC dataframe with the first-merged healthy list
healthy = healthy.merge(healthy_tic2ric, on=['Ticker'], how='left')
# Drop duplicate columns resulted from the merge method
healthy = healthy.drop_duplicates(subset=None, keep='first', inplace=False)

print('\n--------------------- Second Merge Done! -----------------------------\n')

# the second merge results
print('The merge result:')
print('\n\n', healthy.head())



The CUSIP-to-RIC conversion dataframe:

        CUSIP  csp2RIC
0  000361105      AIR
1  000886309      NaN
2  001058205  IWKS.PK
3  020813101      NaN
4  02376R102    AAL.O

---------------------- First Merge Done! -----------------------------

The merge result:

  Identifier                      Company Data Deletion Date Deletion Reason  \
0     001004                     AAR Corp                  .             NaN   
1     001013  ADC Telecommunications Inc.         12/10/2010             1.0   
2     001021             AFP Imaging Corp         09/15/2014             7.0   
3     001034                Alpharma Inc.         12/31/2008             1.0   
4     001045  American Airlines Group Inc                  .             NaN   

  Ticker      CUSIP         CIK          ISIN ISINc  csp2RIC  
0    AIR  000361105  0000001750  US0003611052     0      AIR  
1   ADCT  000886309  0000061478  US0008863096     1      NaN  
2   IWKS  001058205  0000319126  US0010582056     0  IWKS.PK  
3

In [32]:
# ----------------------------------------------------------------------
# merge the two RIC columns
# ----------------------------------------------------------------------
# add a single column for RIC, and
healthy['RIC'] = ''
# a column to indicate there was inconsistency in converting symbols
healthy['RICc'] = ''

for index, row in healthy.iterrows():
    # in case of conflicts
    if row['csp2RIC'] != row['tic2RIC']:
        
        # in case csp2RIC is NAN
        if row['csp2RIC'] != row['csp2RIC']:
            # in case both are NAN values
            if row['tic2RIC'] != row['tic2RIC']:
                # enter NAN as the ISIN value
                row['RIC'] = row['csp2RIC']
                # but no real inconsistency
                row['RICc'] = 0
                
            # in case only csp2RIC is NAN
            else:
                # fill RIC with tic2RIC
                row['RIC'] = row['tic2RIC']
                # and mark it as an inconsistency
                row['RICc'] = 1
        
        # in case csp2RIC is non-NAN value
        else:
            # in case tic2RIC is NAN
            if row['tic2RIC'] != row['tic2RIC']:
                # fill RIC with csp2RIC
                row['RIC'] = row['csp2RIC']
                # and mark it as an inconsistency
                row['RICc'] = 1
            
            # in case both are non-NAN values
            else:
                # fill RIC with both
                row['RIC'] = str(row['csp2RIC']) + ' - ' + str(row['tic2RIC'])
                # and mark it as a serious inconsistency
                row['RICc'] = 3
            
    # in case of consistency        
    else:
        # fill RIC with the value
        row['RIC'] = row['csp2RIC']
        # and it's consistent
        row['RICc'] = 0

        
# check the number of inconsistency cases
inconsistency = len(healthy) - healthy['RICc'].value_counts()[0]
print(inconsistency, 'case(s) of inconsistency!')
# check for serious cases of inconsistency
if 3 in healthy['RICc'].value_counts().index:
    print(healthy['RICc'].value_counts()[3], 'are serious!')
else:
    print('But none is serious.')
    
# check the number of successful conversion to RIC
print('\nAnd now we have', len(healthy) - healthy['RIC'].isna().sum(), 'bankrupt company with RIC code.')


# remove the extra *RIC columns
healthy = healthy.drop(['csp2RIC', 'tic2RIC'], axis=1)


healthy.head()


8805 case(s) of inconsistency!
146 are serious!

And now we have 10493 bankrupt company with RIC code.


,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK,ISIN,ISINc,RIC,RICc
0,001004,AAR Corp,.,NaN,AIR,000361105,0000001750,US0003611052,0,AIR,0
1,001013,ADC Telecommunications Inc.,12/10/2010,1.0,ADCT,000886309,0000061478,US0008863096,1,NaN,0
2,001021,AFP Imaging Corp,09/15/2014,7.0,IWKS,001058205,0000319126,US0010582056,0,IWKS.PK,1
3,001034,Alpharma Inc.,12/31/2008,1.0,ALO.2,020813101,0000730469,US0208131013,1,NaN,0
4,001045,American Airlines Group Inc,.,NaN,AAL,02376R102,0000006201,US02376R1023,0,AAL.O - AAL.Z,3


****

In [33]:
# save the final csv files of bankrupt and healthy companies
bankrupt.to_csv('0.bankrupt_list.csv')
healthy.to_csv('0.healthy_list.csv')
